Notes
* It appears that parents of siblings are simple averages of the siblings.

In [136]:
import pandas as pd
import datetime as dt
from pathlib import Path

DEBUG = False

datapath = Path("../datasets/prices")
writepath = Path("../datasets/prices-preclean")

def fix_month_year_ordering(df):
    cols = df.columns.tolist()
    date_month_cols = sorted([dt.datetime.strptime(x, "%Y %b") for x in cols[2:]])
    date_month_cols = [dt.date.strftime(x, "%Y %b") for x in date_month_cols]
    cols[2:] = date_month_cols
    df = df[cols]
    return df

files = list(datapath.glob("*.csv"))
file = files[0]
df = pd.read_csv(file, index_col=0)
df.drop(list(df.filter(regex="Ave|Annual|2024")), axis=1, inplace=True)  # Remove Ave, Annual, and 2024 Columns
df = fix_month_year_ordering(df)
print(df.isna().any(axis=1).sum())
 


40120


In [20]:
# We noticed that there are a lot of "<month> Annual" values that are NA 
# We explore this
for year in range(2018, 2025):
    # print(df.filter(regex=f"{year} Annual"))
    year_na = df.filter(regex=f"{year} Annual").isnull().any(axis=1).sum()
    print(f"Dataset filtered to {year} has {year_na} rows with NA values")
# df = df[df.columns.drop(list(df.filter(regex='Annual')))]

# null_count = df.isna().any(axis=1).sum()
# print('Number of rows with null values:', null_count)
# # df.dropna(axis=0, inplace=True)
# output_df = df[(df.drop(["Geolocation", "Commodity"], axis=1) != float(0)).any(axis=1)]
# output_df.to_csv("datasets/clean/dummy.csv")

Dataset filtered to 2018 has 6018 rows with NA values
Dataset filtered to 2019 has 6018 rows with NA values
Dataset filtered to 2020 has 6018 rows with NA values
Dataset filtered to 2021 has 6018 rows with NA values
Dataset filtered to 2022 has 6018 rows with NA values
Dataset filtered to 2023 has 6018 rows with NA values
Dataset filtered to 2024 has 6018 rows with NA values


In [ ]:
import json
df_grouped = df.groupby("Commodity Description")
geolocs = df_grouped.get_group("0 - ALL ITEMS").Geolocation
geolocs = list(geolocs)
geolocs_rels = {}
natl = "PHILIPPINES"
region = None
while len(geolocs) != 0:
    loc = str(geolocs.pop(0))
    if (loc == natl):
        # Natl case
        geolocs_rels.update({natl:{}})
    elif (loc.startswith("....")):
        # Province or HUC case (discard if HUC)
        if "City" in loc:
            continue
        province = loc.strip(".")
        geolocs_rels[natl][region].append(province)
    elif (loc.startswith("..")):
        # Region case
        region = loc.strip(".")
        geolocs_rels[natl].update({region:[]})
        
with open('region_provinces.json', 'w') as fp:
    json.dump(geolocs_rels, fp)

In [16]:
import pandas as pd
from pathlib import Path

datapath = Path("../datasets/New")
for file in datapath.iterdir():
    print(file)


..\datasets\New\2M4ACP09 (1).csv
..\datasets\New\2M4ACP09 (2).csv
..\datasets\New\2M4ACP09 (3).csv
..\datasets\New\2M4ACP09 (4).csv
..\datasets\New\2M4ACP09 (5).csv
..\datasets\New\2M4ACP09 (6).csv
..\datasets\New\2M4ACP09 (7).csv
..\datasets\New\2M4ACP09 (8).csv
